<b>RATING PREDICTION</b><br>
--
AUTHOR : 
* Nub-T
* sammyon7

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hamorahadi","key":"55ee72819a572a66fca8455d7d869215"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jvanelteren/boardgamegeek-reviews

100% 451M/451M [00:04<00:00, 110MB/s] 



In [5]:
!kaggle datasets download -d danielwillgeorge/glove6b100dtxt

100% 131M/131M [00:01<00:00, 76.8MB/s]



In [6]:
!ls

boardgamegeek-reviews.zip  glove6b100dtxt.zip  kaggle.json  sample_data


In [7]:
import os
os.listdir()

['.config',
 'kaggle.json',
 'boardgamegeek-reviews.zip',
 'glove6b100dtxt.zip',
 'sample_data']

In [9]:
import torch
import torchtext
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.nn import functional as F

In [10]:
import re
import random
from sklearn.model_selection import train_test_split

In [11]:
import zipfile
zip_ref = zipfile.ZipFile('boardgamegeek-reviews.zip', 'r')
zip_ref.extractall('boardgamegeek-reviews')
zip_ref.close()

In [12]:
import zipfile
zip_ref = zipfile.ZipFile('glove6b100dtxt.zip', 'r')
zip_ref.extractall('glove6b100dtxt')
zip_ref.close()

In [15]:
import pandas as pd

data_df = pd.read_csv('boardgamegeek-reviews/bgg-15m-reviews.csv',usecols=[ "rating", "comment"])[["comment", "rating"]]

In [16]:
data_df.head(10)

,comment,rating
0,NaN,10.0
1,Hands down my favorite new game of BGG CON 200...,10.0
2,I tend to either love or easily tire of co-op ...,10.0
3,NaN,10.0
4,This is an amazing co-op game. I play mostly ...,10.0
5,Hey! I can finally rate this game I've been pl...,10.0
6,NaN,10.0
7,NaN,10.0
8,Love it- great fun with my son. 2 plays so far...,10.0
9,"Fun, fun game. Strategy is required, but defin...",10.0


In [17]:
# Drop the missing value

df = data_df[data_df['comment'].notna()]
df.head(10)

,comment,rating
1,Hands down my favorite new game of BGG CON 200...,10.0
2,I tend to either love or easily tire of co-op ...,10.0
4,This is an amazing co-op game. I play mostly ...,10.0
5,Hey! I can finally rate this game I've been pl...,10.0
8,Love it- great fun with my son. 2 plays so far...,10.0
9,"Fun, fun game. Strategy is required, but defin...",10.0
11,I don't know how they balanced this thing. Ab...,10.0
12,Best collaborative multi-player game out there...,10.0
13,(25+ plays) Who can play? Family game for ...,10.0
16,"A thematic, brisk game that is well balanced a...",10.0


In [18]:
df['rating'] = df['rating'].apply(lambda x: round(x))
df['comment'] = df['comment'].apply(lambda x: x.lower())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
# Let's see the unique value!
df['rating'].unique()

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [20]:
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,comment,rating
0,"great tv show, poor game. play it more for the...",5
1,here i root. really light asymmetrical wargame...,5
2,01/2009:6 a light game but highly dependent on...,6
3,"i like the game, and the minis are for the mos...",7
4,not a bad game but seems fairly derivative to ...,6
5,"very interesting game. relatively simple, and...",7
6,"it's good, but i need to play it more, to rais...",7
7,satisfying to make massive combos in order to ...,9
8,nice engine builder. the option of going low t...,7
9,"fun game, lots of luck but plays fast. the mag...",7


In [21]:
pattern = re.compile("[^a-zA-Z ]+")
df["comment"] = df['comment'].map(lambda x: pattern.sub('', x))
df.head(10)

,comment,rating
0,great tv show poor game play it more for the c...,5
1,here i root really light asymmetrical wargame ...,5
2,a light game but highly dependent on luck run...,6
3,i like the game and the minis are for the most...,7
4,not a bad game but seems fairly derivative to ...,6
5,very interesting game relatively simple and y...,7
6,its good but i need to play it more to raise t...,7
7,satisfying to make massive combos in order to ...,9
8,nice engine builder the option of going low to...,7
9,fun game lots of luck but plays fast the magic...,7


In [22]:
# Drop the comment with len <= 10
df = df[df['comment'].map(len) > 10]
print(len(df))
df = df.reset_index(drop=True)

2774889


In [23]:
df.head(10)

,comment,rating
0,great tv show poor game play it more for the c...,5
1,here i root really light asymmetrical wargame ...,5
2,a light game but highly dependent on luck run...,6
3,i like the game and the minis are for the most...,7
4,not a bad game but seems fairly derivative to ...,6
5,very interesting game relatively simple and y...,7
6,its good but i need to play it more to raise t...,7
7,satisfying to make massive combos in order to ...,9
8,nice engine builder the option of going low to...,7
9,fun game lots of luck but plays fast the magic...,7


In [24]:
df['comment'].map(len).max()

25083

In [25]:
training_df, testing_df = df.loc[:0.5*len(df)], df.loc[0.75*len(df):]

In [26]:
del data_df
del df

In [29]:
training_df.to_csv("training.csv", index=False)
testing_df.to_csv("testing.csv", index=False)

In [30]:
# Here we will use Torch tokenizer
tokenizer = lambda x: x.split()

In [32]:
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True, batch_first=True, fix_length=200)
LABEL = torchtext.data.LabelField(dtype=torch.float)

In [33]:
fields = [('comment',TEXT),('rating', LABEL)]

In [36]:
train_data = torchtext.data.TabularDataset("training.csv","csv", fields, skip_header=True)

In [37]:
test_data = torchtext.data.TabularDataset("testing.csv","csv", fields, skip_header=True)
train_data.examples[0].comment, train_data.examples[0].rating

del training_df
del testing_df

In [39]:
# Create word embeddings!

TEXT.build_vocab(train_data, vectors=torchtext.vocab.Vectors("glove6b100dtxt/glove.6B.100d.txt", cache = 'working/vector_cache'))
LABEL.build_vocab(train_data)


100%|█████████▉| 399789/400000 [00:32<00:00, 21887.62it/s]

In [40]:
word_embeddings = TEXT.vocab.vectors
word_embeddings.shape # Let's see the shape view

torch.Size([438568, 100])

In [41]:
# Split them!
train_data, valid_data = train_data.split()

In [42]:
train_iter, valid_iter, test_iter = torchtext.data.BucketIterator.splits((train_data, valid_data, test_data),
                                                               batch_size=32,
                                                               sort_key=lambda x: len(x.comment),
                                                               repeat=False,
                                                               shuffle=True)

In [44]:
vocab_size = len(TEXT.vocab)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(vocab_size, device)

438568 cuda


In [46]:
word_embeddings.shape

torch.Size([438568, 100])

In [47]:
# Save into .pt embedding file!
torch.save(word_embeddings, "word_embeddings.pt")

In [49]:
import dill

with open("TEXT.Field", "wb") as f:
    dill.dump(TEXT, f)

In [51]:
# Define the model!

class ClassifierModel(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
        super(ClassifierModel, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)  
        self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) 
        self.lstm = nn.LSTM(embedding_length, hidden_size)
        self.label = nn.Linear(hidden_size, output_size)

    def forward(self, input_sentence, batch_size=None):
        input = self.word_embeddings(input_sentence) 
        input = input.permute(1, 0, 2) 

        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())

        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        final_output = self.label(final_hidden_state[-1]) 

        return final_output

In [54]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

In [55]:
# Training defined function
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.to(device)
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()

    for idx, batch in enumerate(train_iter):
        text = batch.comment[0]
        target = batch.rating
        target = torch.autograd.Variable(target).long()

        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] != 32):
            continue

        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

# Evaluation defined function
def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.comment[0]

            if (text.size()[0] != 32):
                continue

            target = batch.rating
            target = torch.autograd.Variable(target).long()

            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()

            prediction = model(text)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

In [56]:
batch_size = 32
output_size = 11
hidden_size = 256
embedding_length = 100
model = ClassifierModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [58]:
model

ClassifierModel(
  (word_embeddings): Embedding(438568, 100)
  (lstm): LSTM(100, 256)
  (label): Linear(in_features=256, out_features=11, bias=True)
)

In [59]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 369,419 trainable parameters


In [60]:
learning_rate = 0.001
loss_fn = F.cross_entropy

In [ ]:
for epoch in range(100):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')

In [ ]:
test_sent = "This game is interesting"
test_sent = TEXT.preprocess(test_sent)
test_sent = [[TEXT.vocab.stoi[x] for x in test_sent]]
test_sent = np.asarray(test_sent)
test_sent = torch.LongTensor(test_sent)
test_tensor = Variable(test_sent)
test_tensor = test_tensor.cuda()
model.eval()
output = model(test_tensor, 1)
out = F.softmax(output, 1)
out